In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-beta0'

### Learning from dice sequences
This is a toy problem. Any sequence of 5 throws is rewarded with a particular value based on some prior *truth*. Ten different independent dice are being rewarded at the same time. 

Think of this as 5 consecutive days of trading in 10 different stocks. For each stock, there's one day where buying it will give an immediate reward. That reward is different for every stock and every day, based on the *mask* matrix below.

We hope that the network learns when what stock to trade on what day.

In [3]:
BATCH_SIZE = 3
N_STEPS = 5
N_FEATURES = 10

N_STEPS consecutive samples with N_FEATURES dice. Each sequence of N_STEPS gets a score. E.g. the first die gets a tenth of what the first result was. So, each die (feature) is independent of the others.

In [4]:
x = np.floor(np.random.random([BATCH_SIZE, N_STEPS, N_FEATURES]) * 7).astype(np.float32)
x.shape

(3, 5, 10)

In [5]:
mask = np.zeros((5,10), dtype=np.float32)

for k in range(5):
    mask[k][k]=(k+1)/10.
    mask[k][k+5]=.6-(k+1)/10.

mask

array([[0.1, 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. ],
       [0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0.1]], dtype=float32)

In [6]:
def truth(x):
    logits = np.round(np.sum(x * mask, axis=0), 3)
    e = np.exp(logits)
    return e / np.sum(e)

In [7]:
truth(x[0])

array([0.0427836 , 0.05225601, 0.05225601, 0.06382563, 0.34937814,
       0.12852903, 0.14204656, 0.0387122 , 0.0779568 , 0.05225601],
      dtype=float32)

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import activations

## Heuristic
Try to just mimic the truth function, let it find out the mask values

In [63]:
class Heuristic_TradeModel(tf.keras.Model):
    def __init__(self, units):
        super(Heuristic_TradeModel, self).__init__()
        self.selector = layers.Dense(
            10, activation='relu', 
            kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
        self.denses = [layers.Dense(u, activation='relu') for u in units]
        self.logits = layers.Dense(10, activation = None)
        
    def call(self, inputs):
        out = tf.cast(inputs / 100, dtype=tf.float32)
        out = tf.reshape(out, shape=[-1, 50])
        out = self.selector(out)
        for layer in self.denses:
            out = layer(out) 
        out = self.logits(out)
        return out
    
    def portfolio(self, inputs):
        return activations.softmax(self.call(inputs))

model = Heuristic_TradeModel([64])
x = np.floor(np.random.random(
    [BATCH_SIZE, N_STEPS, N_FEATURES]) * 7).astype(np.float32)

_ = model(x)
model.summary()

Model: "dense__trade_model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             multiple                  510       
_________________________________________________________________
dense_36 (Dense)             multiple                  704       
_________________________________________________________________
dense_37 (Dense)             multiple                  650       
Total params: 1,864
Trainable params: 1,864
Non-trainable params: 0
_________________________________________________________________


## LSTM and Dense Model 
...just don't work!!

In [63]:
class Dense_TradeModel(tf.keras.Model):
    def __init__(self, units):
        super(Dense_TradeModel, self).__init__()
        self.selector = layers.Dense(
            10, activation='relu', 
            kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
        self.denses = [layers.Dense(u, activation='relu') for u in units]
        self.logits = layers.Dense(10, activation = None)
        
    def call(self, inputs):
        out = tf.cast(inputs / 100, dtype=tf.float32)
        out = tf.reshape(out, shape=[-1, 50])
        out = self.selector(out)
        for layer in self.denses:
            out = layer(out) 
        out = self.logits(out)
        return out
    
    def portfolio(self, inputs):
        return activations.softmax(self.call(inputs))

model = Dense_TradeModel([64])
x = np.floor(np.random.random(
    [BATCH_SIZE, N_STEPS, N_FEATURES]) * 7).astype(np.float32)

_ = model(x)
model.summary()

Model: "dense__trade_model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             multiple                  510       
_________________________________________________________________
dense_36 (Dense)             multiple                  704       
_________________________________________________________________
dense_37 (Dense)             multiple                  650       
Total params: 1,864
Trainable params: 1,864
Non-trainable params: 0
_________________________________________________________________


In [26]:
class LSTM_TradeModel(tf.keras.Model):
    def __init__(self, n_neurons):
        super(LSTM_TradeModel, self).__init__()
        self.lstm = layers.LSTM(
            n_neurons, 
            input_shape=[None, 5, 10],
            return_sequences=False)
        self.logits = layers.Dense(10, activation = None)
        
    def call(self, inputs):
        out = tf.cast(inputs / 100, dtype=tf.float32)
        out = self.lstm(out) 
        out = self.logits(out)
        return out
    
    def portfolio(self, inputs):
        return activations.softmax(self.call(inputs))

model = LSTM_TradeModel(128)
x = np.floor(np.random.random(
    [BATCH_SIZE, N_STEPS, N_FEATURES]) * 7).astype(np.float32)

_ = model(x)
model.summary()

Model: "lstm__trade_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  71168     
_________________________________________________________________
dense_25 (Dense)             multiple                  1290      
Total params: 72,458
Trainable params: 72,458
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.portfolio(x).numpy()

array([[0.09941742, 0.09997836, 0.10062797, 0.10010673, 0.09946124,
        0.10159553, 0.09931004, 0.09972215, 0.09942178, 0.10035885],
       [0.09942142, 0.09965046, 0.1002036 , 0.10077167, 0.09883868,
        0.10254937, 0.09884752, 0.09973002, 0.09971128, 0.1002759 ],
       [0.09942549, 0.09944106, 0.10058582, 0.10089849, 0.09837616,
        0.10421751, 0.09801126, 0.09959725, 0.09978559, 0.09966137]],
      dtype=float32)

In [65]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=loss, optimizer=optimizer)

In [66]:
def new_data_set(n_steps, n_features, total_size,
                 batch_size, buffer_size=1000):
    def _generator():
        x = np.floor( np.random.random(
                [n_steps, n_features]) * 7).astype(np.float32)
        y = truth(x)
        for _ in range(total_size):
            yield x,y

    inputs = tf.data.Dataset.from_generator(
        _generator, output_types = (tf.float32, tf.float32))

    inputs = inputs.shuffle(buffer_size).batch(batch_size)

    return inputs

In [67]:
dataset = new_data_set(5, 10, 1000, 1)

In [68]:
next(iter(dataset))

(<tf.Tensor: id=20177, shape=(1, 5, 10), dtype=float32, numpy=
 array([[[0., 6., 6., 4., 2., 3., 0., 4., 0., 3.],
         [0., 6., 6., 2., 4., 5., 0., 3., 5., 6.],
         [1., 0., 3., 2., 6., 0., 1., 5., 1., 5.],
         [2., 4., 5., 3., 5., 6., 3., 4., 0., 5.],
         [2., 6., 3., 2., 1., 6., 6., 4., 6., 4.]]], dtype=float32)>,
 <tf.Tensor: id=20178, shape=(1, 10), dtype=float32, numpy=
 array([[0.04131589, 0.1371736 , 0.1016207 , 0.1371736 , 0.06811839,
         0.18516497, 0.04131589, 0.18516497, 0.04131589, 0.06163607]],
       dtype=float32)>)

In [72]:
dataset = new_data_set(5, 10, 100000, 5000)
model.fit(dataset, epochs=20)

Epoch 1/20
20/20 [==============================] - 5s 233ms/step - loss: 2.2046
Epoch 2/20
20/20 [==============================] - 5s 233ms/step - loss: 2.1954
Epoch 3/20
20/20 [==============================] - 5s 228ms/step - loss: 2.2771
Epoch 4/20
20/20 [==============================] - 5s 232ms/step - loss: 2.3941
Epoch 5/20
20/20 [==============================] - 5s 230ms/step - loss: 2.2210
Epoch 6/20
20/20 [==============================] - 5s 234ms/step - loss: 2.1072
Epoch 7/20
20/20 [==============================] - 5s 231ms/step - loss: 2.2139
Epoch 8/20
20/20 [==============================] - 5s 234ms/step - loss: 2.0413
Epoch 9/20
20/20 [==============================] - 5s 237ms/step - loss: 2.4155
Epoch 10/20
20/20 [==============================] - 5s 231ms/step - loss: 2.0789
Epoch 11/20
20/20 [==============================] - 5s 231ms/step - loss: 2.2822
Epoch 12/20
20/20 [==============================] - 5s 233ms/step - loss: 2.2689
Epoch 13/20
20/20 [======

In [73]:
model.evaluate(new_data_set(5, 10, 100, 100))

      1/Unknown - 0s 15ms/step - loss: 2.1762

2.1762139797210693

In [77]:
dataset = new_data_set(5, 10, 1000, 1)
x, y = next(iter(dataset))
x, y

(<tf.Tensor: id=27496, shape=(1, 5, 10), dtype=float32, numpy=
 array([[[3., 6., 6., 0., 4., 2., 3., 4., 1., 2.],
         [2., 2., 5., 4., 6., 1., 2., 4., 4., 1.],
         [1., 3., 5., 6., 2., 2., 4., 0., 2., 1.],
         [0., 0., 2., 2., 5., 3., 3., 1., 1., 3.],
         [5., 0., 3., 5., 1., 2., 2., 3., 5., 5.]]], dtype=float32)>,
 <tf.Tensor: id=27497, shape=(1, 10), dtype=float32, numpy=
 array([[0.06745388, 0.07454807, 0.22395477, 0.11121264, 0.08238835,
         0.13583542, 0.11121264, 0.04997106, 0.06103479, 0.08238835]],
       dtype=float32)>)

In [78]:
model.portfolio(x)

<tf.Tensor: id=27518, shape=(1, 10), dtype=float32, numpy=
array([[0.04695836, 0.05248175, 0.07947842, 0.12919323, 0.19288217,
        0.11984455, 0.16159499, 0.09934296, 0.06611667, 0.05210704]],
      dtype=float32)>

In [ ]:
np.